<a href="https://colab.research.google.com/github/mahmudresin/Senior-Design-Project-499/blob/main/For%20qnn%20setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'experimental:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5847681%2F9588433%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241013%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241013T121241Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4790da4b5b5c0251b5b0cbe2e52aab2e9056a1a7e58a7016ec8e554f3d4ed0a7e5d8bcd0aef2a1975082ae098d4d019e84f4d5c9f60234e97e6848a4ac95031967bf6c8e1c3335d65d888ea3814381740b9679f450711e20b1493bc13a64dea231a6a0d4b2447b49bc217a847418041dfee6d62b46b3c02327d2b4aa6c3aae056507ebae2b0abebd96fd46277a3fde1fa13ac5349043035c61e73a2bd0dc523221143880dc605c7a64e682d942a478375162fcd791dc25c38483cfd580bbaa2da21ade5624ade32644fdba06b569cde81ef7f0ce4d889f4b1fe3d5aa3be1d3201f0dbb562544beed0e07e64ac3f582a13e3f41fdaa924f1e915bf00445163715'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 649517303 bytes downloaded
Downloaded and uncompressed: experimental
Data source import complete.


In [ ]:
# Import necessary libraries
from fastai.imports import *
from fastai.vision.all import *
import PIL
import os
import torch  # Import torch for GPU management

# Function to enable GPU if available
def enable_gpu_if_available():
    if torch.cuda.is_available():
        print("GPU is available.")
        return torch.device('cuda')
    else:
        print("GPU is not available; using CPU instead.")
        return torch.device('cpu')

# Function to cleanup GPU cache
def cleanup_gpu_cache():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()  # Free up unused memory on the GPU
        print("GPU cache cleaned.")
    else:
        print("No GPU found; nothing to clean.")

# Custom utility functions (make sure utils directory is correctly set up)
try:
    from utils.utils import *
except ModuleNotFoundError as e:
    print(f"Error: {e}. Please check if the 'utils' module is available in your path.")

# Ensure proper output display in Jupyter
from IPython.display import clear_output, DisplayHandle

def update_patch(self, obj):
    clear_output(wait=True)
    self.display(obj)

# Environment variable setup for Weights & Biases (wandb)
os.environ["ENABLE_WANDB"] = "ON"

if os.getenv("ENABLE_WANDB"):
    g_ENABLE_WANDB = True
    os.environ["WANDB_NOTEBOOK_NAME"] = "ai4mars.ipynb"
    import wandb
    from fastai.callback.wandb import *

    wandb.login()
    wandb.init(project="AI4Mars")
else:
    g_ENABLE_WANDB = False

# Update the DisplayHandle with the custom output patch
DisplayHandle.update = update_patch

# Enable GPU if available
g_DEVICE = enable_gpu_if_available()
cleanup_gpu_cache()


Setup the paths to our images and labels.

In [ ]:
# Define the base dataset path
dataset_path = Path('/kaggle/input/experimental/Experimental Data')

# Define the paths for images and masks
IMAGES_PATH = Path(dataset_path / "images" )
MASK_PATH_TRAIN = Path(dataset_path /  "train")
MASK_PATH_TEST = Path(dataset_path /  "test" )

#len(IMAGES_PATH.ls()), len(MASK_PATH_TRAIN.ls()), len(MASK_PATH_TEST.ls())

In [ ]:
# Import necessary libraries
from fastai.imports import *
from fastai.vision.all import *
import PIL
import os
import torch
from pathlib import Path  # Import Path for handling file paths

# Define the path to your dataset
dataset_path = Path('/path/to/mars_dataset')  # Replace with the actual path to your dataset

# Define image and mask paths
IMAGES_PATH = Path(dataset_path / "images")
MASK_PATH_TRAIN = Path(dataset_path / "train")
MASK_PATH_TEST = Path(dataset_path / "test")

# Function to prepare the dataset
def prepare_dataset(images_path, mask_path_train, mask_path_test):
    # This is a placeholder implementation. You should implement the logic to prepare your dataset here.
    images_train = []  # Logic to load training images
    images_test = []   # Logic to load test images
    images_unused = [] # Logic to load unused images

    # Example: Loading images (you need to implement this according to your dataset structure)
    # images_train = [Image.open(img) for img in images_path.glob('train/*.png')]  # Example code
    # images_test = [Image.open(img) for img in images_path.glob('test/*.png')]    # Example code

    return images_train, images_test, images_unused

# Function to enable GPU if available
def enable_gpu_if_available():
    if torch.cuda.is_available():
        print("GPU is available.")
        return torch.device('cuda')
    else:
        print("GPU is not available; using CPU instead.")
        return torch.device('cpu')

# Function to cleanup GPU cache
def cleanup_gpu_cache():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()  # Free up unused memory on the GPU
        print("GPU cache cleaned.")
    else:
        print("No GPU found; nothing to clean.")

# Custom utility functions (make sure utils directory is correctly set up)
try:
    from utils.utils import *
except ModuleNotFoundError as e:
    print(f"Error: {e}. Please check if the 'utils' module is available in your path.")

# Ensure proper output display in Jupyter
from IPython.display import clear_output, DisplayHandle

def update_patch(self, obj):
    clear_output(wait=True)
    self.display(obj)

# Environment variable setup for Weights & Biases (wandb)
os.environ["ENABLE_WANDB"] = "ON"

if os.getenv("ENABLE_WANDB"):
    g_ENABLE_WANDB = True
    os.environ["WANDB_NOTEBOOK_NAME"] = "ai4mars.ipynb"
    import wandb
    from fastai.callback.wandb import *

    wandb.login()
    wandb.init(project="AI4Mars")
else:
    g_ENABLE_WANDB = False

# Update the DisplayHandle with the custom output patch
DisplayHandle.update = update_patch

# Enable GPU if available
g_DEVICE = enable_gpu_if_available()
cleanup_gpu_cache()

# Prepare dataset
IMAGES_PATH_TRAIN, IMAGES_PATH_TEST, IMAGES_PATH_UNUSED = prepare_dataset(
    IMAGES_PATH, MASK_PATH_TRAIN, MASK_PATH_TEST
)


In [ ]:
!pip install pennylane
from fastai.vision.all import *
import pennylane as qml
from pennylane import numpy as np

dataset_path = Path('/kaggle/input/experimental/Experimental Data')
# Define image and mask paths
IMAGES_PATH = Path(dataset_path / "images")
MASK_PATH_TRAIN = Path(dataset_path / "train")
MASK_PATH_TEST = Path(dataset_path / "test")

# Load images and masks
def load_images_and_masks(image_path, mask_path, size=(128, 128)):
    images = get_image_files(image_path)
    masks = get_image_files(mask_path)

    # Ensure sorted order (in case filenames are not in order)
    images.sort()
    masks.sort()

    image_data = []
    mask_data = []

    # Loop over each image and mask to resize and add to list
    for img_file, mask_file in zip(images, masks):
        img = PILImage.create(img_file)
        mask = PILImage.create(mask_file)

        # Resize
        img = img.resize(size)
        mask = mask.resize(size)

        # Convert to tensor and normalize
        img_tensor = tensor(img) / 255.0
        mask_tensor = tensor(mask) / 255.0

        image_data.append(img_tensor)
        mask_data.append(mask_tensor)

    return torch.stack(image_data), torch.stack(mask_data)

train_images, train_masks = load_images_and_masks(IMAGES_PATH, MASK_PATH_TRAIN)


In [ ]:
# Define the number of qubits (depending on image size)
n_qubits = 4  # Can be adjusted

# Quantum device
dev = qml.device("default.qubit", wires=n_qubits)

# Quantum circuit
@qml.qnode(dev, interface="torch")
def quantum_circuit(inputs):
    # Encode classical data into quantum states
    for i in range(n_qubits):
        qml.RX(inputs[i], wires=i)  # Rotation along X-axis (can be adjusted)

    # Apply quantum operations
    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i+1])

    # Measurement
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

# Example of encoding data (you would typically pass the image data here)
data_example = torch.tensor([0.1, 0.2, 0.3, 0.4], requires_grad=True)
output = quantum_circuit(data_example)
print(output)


In [ ]:
import torch.nn as nn

class HybridQuantumClassicalModel(nn.Module):
    def __init__(self, n_qubits, n_output):
        super(HybridQuantumClassicalModel, self).__init__()
        self.n_qubits = n_qubits
        self.n_output = n_output

        # Define a classical fully connected layer
        self.fc1 = nn.Linear(n_qubits, 64)
        self.fc2 = nn.Linear(64, n_output)

    def forward(self, x):
        # Assume x is a batch of quantum-encoded inputs
        quantum_output = []

        # Pass each input through the quantum circuit
        for i in range(x.shape[0]):
            q_out = quantum_circuit(x[i])
            quantum_output.append(q_out)

        quantum_output = torch.stack(quantum_output)

        # Pass the quantum circuit output through classical layers
        x = torch.relu(self.fc1(quantum_output))
        x = self.fc2(x)

        return x

# Example usage:
model = HybridQuantumClassicalModel(n_qubits=n_qubits, n_output=1)  # Output 1 for regression/classification
output = model(data_example.unsqueeze(0))  # Adding a batch dimension
print(output)


In [ ]:
import torch
import torch.nn as nn
import pennylane as qml

class HybridQuantumClassicalModel(nn.Module):
    def __init__(self, n_qubits, n_output):
        super(HybridQuantumClassicalModel, self).__init__()
        self.n_qubits = n_qubits
        self.n_output = n_output

        # Define a classical fully connected layer
        self.fc1 = nn.Linear(n_qubits, 64)
        self.fc2 = nn.Linear(64, n_output)

    def forward(self, x):
        # Assume x is a batch of quantum-encoded inputs
        quantum_output = []

        # Pass each input through the quantum circuit
        for i in range(x.shape[0]):
            q_out = quantum_circuit(x[i])  # Returns a tensor from the quantum circuit
            q_out = torch.tensor(q_out, dtype=torch.float32)  # Ensure it's a tensor
            quantum_output.append(q_out)

        # Stack the quantum outputs into a single tensor
        quantum_output = torch.stack(quantum_output)

        # Pass the quantum circuit output through classical layers
        x = torch.relu(self.fc1(quantum_output))
        x = self.fc2(x)

        return x

# Quantum circuit
dev = qml.device("default.qubit", wires=4)

@qml.qnode(dev, interface="torch")
def quantum_circuit(inputs):
    # Encode classical data into quantum states
    for i in range(4):  # Assuming 4 qubits
        qml.RX(inputs[i], wires=i)

    # Apply quantum operations (for example, CNOT gates)
    for i in range(3):
        qml.CNOT(wires=[i, i+1])

    # Return the expectation values of the Pauli-Z operator for each qubit
    return [qml.expval(qml.PauliZ(i)) for i in range(4)]  # Returns a list of 4 values

# Example usage
n_qubits = 4  # Number of qubits
model = HybridQuantumClassicalModel(n_qubits=n_qubits, n_output=1)

# Example input data (with a batch dimension)
data_example = torch.tensor([[0.1, 0.2, 0.3, 0.4]], requires_grad=True)

# Forward pass through the model
output = model(data_example)
print(output)


In [ ]:
import torch.nn as nn

class HybridQuantumClassicalModel(nn.Module):
    def __init__(self, n_qubits, n_output):
        super(HybridQuantumClassicalModel, self).__init__()
        self.n_qubits = n_qubits
        self.n_output = n_output

        # Define a classical fully connected layer
        self.fc1 = nn.Linear(n_qubits, 64)
        self.fc2 = nn.Linear(64, n_output)

    def forward(self, x):
        # Assume x is a batch of quantum-encoded inputs
        quantum_output = []

        # Pass each input through the quantum circuit
        for i in range(x.shape[0]):
            q_out = quantum_circuit(x[i])
            quantum_output.append(q_out)

        quantum_output = torch.stack(quantum_output)

        # Pass the quantum circuit output through classical layers
        x = torch.relu(self.fc1(quantum_output))
        x = self.fc2(x)

        return x

# Example usage:
model = HybridQuantumClassicalModel(n_qubits=n_qubits, n_output=1)  # Output 1 for regression/classification
output = model(data_example.unsqueeze(0))  # Adding a batch dimension
print(output)


In [ ]:
import torch
import torch.nn as nn
import pennylane as qml

class HybridQuantumClassicalModel(nn.Module):
    def __init__(self, n_qubits, n_output):
        super(HybridQuantumClassicalModel, self).__init__()
        self.n_qubits = n_qubits
        self.n_output = n_output

        # Define a classical fully connected layer
        self.fc1 = nn.Linear(n_qubits, 64)
        self.fc2 = nn.Linear(64, n_output)

    def forward(self, x):
        # Assume x is a batch of quantum-encoded inputs
        quantum_output = []

        # Pass each input through the quantum circuit
        for i in range(x.shape[0]):
            q_out = quantum_circuit(x[i])
            q_out = torch.tensor(q_out, dtype=torch.float32)
            quantum_output.append(q_out)

        # Stack the quantum outputs into a single tensor
        quantum_output = torch.stack(quantum_output)

        # Pass the quantum circuit output through classical layers
        x = torch.relu(self.fc1(quantum_output))
        x = self.fc2(x)

        return x

# Define the quantum circuit
dev = qml.device("default.qubit", wires=4)

@qml.qnode(dev, interface="torch")
def quantum_circuit(inputs):
    # Encode classical data into quantum states
    for i in range(4):  # Assuming 4 qubits
        qml.RX(inputs[i], wires=i)  # Use RX gate for rotation on each qubit

    # Apply quantum operations (for example, CNOT gates)
    for i in range(3):
        qml.CNOT(wires=[i, i+1])

    # Return the expectation values of the Pauli-Z operator for each qubit
    return [qml.expval(qml.PauliZ(i)) for i in range(4)]  # Return a list of 4 values

# Example usage:
n_qubits = 4  # Number of qubits in the quantum circuit
model = HybridQuantumClassicalModel(n_qubits=n_qubits, n_output=1)

# Example input data (batch size = 1, matching number of qubits)
data_example = torch.tensor([[0.1, 0.2, 0.3, 0.4]], requires_grad=True)

# Forward pass through the model
output = model(data_example)
print(output)


In [ ]:
import torch.optim as optim

# Define loss and optimizer
criterion = nn.MSELoss()  # You can adjust the loss function depending on the task
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop (simplified)
for epoch in range(10):  # Number of epochs
    model.train()
    optimizer.zero_grad()

    # Forward pass
    output = model(train_images)

    # Compute loss
    loss = criterion(output, train_masks)

    # Backpropagation
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

In [ ]:
import torch.optim as optim
import pennylane as qml
import torch
import torch.nn as nn

# Define loss and optimizer
criterion = nn.MSELoss()  # Adjust depending on your task
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Modify the quantum circuit to ensure it processes one sample at a time
def quantum_circuit(x):
    """Process input through the quantum circuit."""
    qml.BasisState(x, wires=range(n_qubits))
    qml.RX(x[0], wires=0)  # Expecting a scalar for RX
    # Add more quantum gates as needed
    return qml.expval(qml.PauliZ(0))

# Training loop
for epoch in range(10):  # Number of epochs
    model.train()
    optimizer.zero_grad()

    # Loop through each sample in the training data (batch processing)
    quantum_output = []
    for i in range(train_images.shape[0]):
        # Ensure input shape is correct for quantum circuit (e.g., single sample)
        x_sample = train_images[i].squeeze()  # Flatten the image if needed
        x_sample = torch.tensor(x_sample, dtype=torch.float32)

        # Forward pass through the quantum circuit
        q_out = quantum_circuit(x_sample)
        q_out = torch.tensor(q_out, dtype=torch.float32)
        quantum_output.append(q_out)

    # Convert list of quantum outputs to a tensor
    output = torch.stack(quantum_output)

    # Compute loss
    loss = criterion(output, train_masks)

    # Backpropagation
    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")


In [ ]:
import torch
import pennylane as qml

# Set the number of qubits in your quantum circuit
n_qubits = 4  # Example number, adjust based on your task

def quantum_circuit(x):
    """Process input through the quantum circuit."""
    # Flatten the input so it becomes one-dimensional
    x_flat = x.flatten()

    # Ensure the input matches the number of qubits
    if len(x_flat) != n_qubits:
        raise ValueError(f"Input size {len(x_flat)} does not match the number of qubits {n_qubits}")

    # Process through quantum gates
    qml.BasisState(x_flat, wires=range(n_qubits))  # Pass flattened input to BasisState
    qml.RX(x_flat[0], wires=0)  # Example of an RX gate
    # Add more quantum gates as needed

    # Return an expectation value as an output
    return qml.expval(qml.PauliZ(0))

# Assuming you have a batch of samples to process
quantum_output = []

for x_sample in x_train_batch:  # Assuming x_train_batch is your input batch
    # Convert sample to a tensor if needed and flatten it
    x_sample = torch.tensor(x_sample, dtype=torch.float32).flatten()

    # Forward pass through the quantum circuit
    q_out = quantum_circuit(x_sample)
    q_out = torch.tensor(q_out, dtype=torch.float32)
    quantum_output.append(q_out)

quantum_output = torch.stack(quantum_output)  # Combine into a tensor for further processing


In [ ]:
import pennylane as qml
import torch
import torch.nn as nn
import numpy as np

# Define number of qubits based on the input
n_qubits = 128  # Update this based on your quantum circuit's design

# Define the quantum circuit
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def quantum_circuit(x):
    """Process input through the quantum circuit."""
    # Flatten the input so it becomes one-dimensional
    x_flat = x.flatten()

    # Ensure the input matches the number of qubits
    if len(x_flat) != n_qubits:
        raise ValueError(f"Input size {len(x_flat)} does not match the number of qubits {n_qubits}")

    # BasisState preparation
    qml.BasisState(x_flat, wires=range(n_qubits))

    # Example quantum gates (customize as needed)
    qml.RX(x_flat[0], wires=0)  # Rotate qubit 0 around X-axis
    qml.RY(x_flat[1], wires=1)  # Rotate qubit 1 around Y-axis
    # Add more quantum gates as necessary

    # Measure an observable (for example, PauliZ on qubit 0)
    return qml.expval(qml.PauliZ(0))

# Define your PyTorch model
class QuantumNet(nn.Module):
    def __init__(self):
        super(QuantumNet, self).__init__()
        # Classical layers (example: one linear layer)
        self.fc1 = nn.Linear(4, n_qubits)  # Adjust input size and output size

    def forward(self, x):
        quantum_output = []

        # Process each sample in the batch
        for x_sample in x:
            # Ensure the input is a float tensor
            x_sample = torch.tensor(x_sample, dtype=torch.float32)

            # Flatten the input if it has multiple dimensions
            x_sample = x_sample.flatten()

            # Forward pass through the quantum circuit
            q_out = quantum_circuit(x_sample.detach().numpy())  # Use numpy array for Pennylane
            q_out = torch.tensor(q_out, dtype=torch.float32)  # Convert quantum output back to tensor
            quantum_output.append(q_out)

        # Stack the quantum outputs and return
        quantum_output = torch.stack(quantum_output)
        return quantum_output

# Example usage
model = QuantumNet()

# Example batch input (adjust size as needed)
x = torch.rand(2, 128, 128, 3)  # Simulating a batch of 2 samples, with shape (128, 128, 3)

# Forward pass through the model
output = model(x)
print(output)


In [ ]:
import torch
import torch.nn as nn
import pennylane as qml
import torch.nn.functional as F

n_qubits = 128  # Set the number of qubits
dev = qml.device("default.qubit", wires=n_qubits)

# Define a quantum circuit
@qml.qnode(dev)
def quantum_circuit(x):
    """Process input through the quantum circuit."""
    x_flat = x.flatten()

    # Ensure the input matches the number of qubits
    if len(x_flat) != n_qubits:
        raise ValueError(f"Input size {len(x_flat)} does not match the number of qubits {n_qubits}")

    # BasisState preparation
    qml.BasisState(x_flat, wires=range(n_qubits))

    # Example quantum gates (can be modified)
    for i in range(n_qubits):
        qml.RX(x_flat[i], wires=i)

    # Return expectation value of PauliZ on the first qubit
    return qml.expval(qml.PauliZ(0))

# Define a hybrid quantum-classical neural network
class QuantumNet(nn.Module):
    def __init__(self):
        super(QuantumNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 16 * 16, n_qubits)  # Adjust for the flattened size

    def forward(self, x):
        # Preprocess the input
        x = self.pool(F.relu(self.conv1(x)))

        # Resize the image to match the number of qubits (e.g., 16x16)
        x = F.interpolate(x, size=(16, 16))  # Resize to 16x16

        # Flatten the image
        x = x.view(-1, 16 * 16 * 16)  # Match the input size to the fully connected layer
        x = F.relu(self.fc1(x))

        quantum_output = []
        for x_sample in x:
            x_sample = x_sample.flatten().detach().numpy()  # Flatten and convert to numpy for Pennylane

            # Forward pass through the quantum circuit
            q_out = quantum_circuit(x_sample)
            q_out = torch.tensor(q_out, dtype=torch.float32)  # Convert quantum output back to tensor
            quantum_output.append(q_out)

        quantum_output = torch.stack(quantum_output)  # Combine into a tensor
        return quantum_output

# Create an instance of the model
model = QuantumNet()

# Simulate input data
x = torch.rand(2, 128, 128, 3)  # Simulating a batch of 2 samples, with shape (128, 128, 3)

# Forward pass through the model
output = model(x)
print(output)


In [ ]:
import torch
import torch.nn as nn
import pennylane as qml
import torch.nn.functional as F

n_qubits = 128  # Set the number of qubits
dev = qml.device("default.qubit", wires=n_qubits)

# Define a quantum circuit
@qml.qnode(dev)
def quantum_circuit(x):
    """Process input through the quantum circuit."""
    x_flat = x.flatten()

    # Ensure the input matches the number of qubits
    if len(x_flat) != n_qubits:
        raise ValueError(f"Input size {len(x_flat)} does not match the number of qubits {n_qubits}")

    # BasisState preparation
    qml.BasisState(x_flat, wires=range(n_qubits))

    # Example quantum gates (can be modified)
    for i in range(n_qubits):
        qml.RX(x_flat[i], wires=i)

    # Return expectation value of PauliZ on the first qubit
    return qml.expval(qml.PauliZ(0))

# Define a hybrid quantum-classical neural network
class QuantumNet(nn.Module):
    def __init__(self):
        super(QuantumNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 16 * 16, n_qubits)  # Adjust for the flattened size

    def forward(self, x):
        # Permute the input to the correct format (batch_size, channels, height, width)
        x = x.permute(0, 3, 1, 2)

        # Preprocess the input
        x = self.pool(F.relu(self.conv1(x)))

        # Resize the image to match the number of qubits (e.g., 16x16)
        x = F.interpolate(x, size=(16, 16))  # Resize to 16x16

        # Flatten the image
        x = x.view(-1, 16 * 16 * 16)  # Match the input size to the fully connected layer
        x = F.relu(self.fc1(x))

        quantum_output = []
        for x_sample in x:
            x_sample = x_sample.flatten().detach().numpy()  # Flatten and convert to numpy for Pennylane

            # Forward pass through the quantum circuit
            q_out = quantum_circuit(x_sample)
            q_out = torch.tensor(q_out, dtype=torch.float32)  # Convert quantum output back to tensor
            quantum_output.append(q_out)

        quantum_output = torch.stack(quantum_output)  # Combine into a tensor
        return quantum_output

# Create an instance of the model
model = QuantumNet()

# Simulate input data
x = torch.rand(2, 128, 128, 3)  # Simulating a batch of 2 samples, with shape (128, 128, 3)

# Forward pass through the model
output = model(x)
print(output)


In [ ]:
import torch
import torch.nn as nn
import pennylane as qml
import torch.nn.functional as F

n_qubits = 128  # Set the number of qubits
dev = qml.device("default.qubit", wires=n_qubits)

# Define a quantum circuit
@qml.qnode(dev)
def quantum_circuit(x):
    """Process input through the quantum circuit."""
    x_flat = x.flatten()

    # Ensure the input matches the number of qubits
    if len(x_flat) != n_qubits:
        raise ValueError(f"Input size {len(x_flat)} does not match the number of qubits {n_qubits}")

    # BasisState preparation
    qml.BasisState(x_flat, wires=range(n_qubits))

    # Example quantum gates (can be modified)
    for i in range(n_qubits):
        qml.RX(x_flat[i], wires=i)

    # Return expectation value of PauliZ on the first qubit
    return qml.expval(qml.PauliZ(0))

# Define a hybrid quantum-classical neural network
class QuantumNet(nn.Module):
    def __init__(self):
        super(QuantumNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 16 * 16, n_qubits)  # Adjust for the flattened size

    def forward(self, x):
        # Permute the input to the correct format (batch_size, channels, height, width)
        x = x.permute(0, 3, 1, 2)

        # Preprocess the input
        x = self.pool(F.relu(self.conv1(x)))

        # Resize the image to match the number of qubits (e.g., 16x16)
        x = F.interpolate(x, size=(16, 16))  # Resize to 16x16

        # Flatten the image
        x = x.reshape(-1, 16 * 16 * 16)  # Use reshape instead of view
        x = F.relu(self.fc1(x))

        quantum_output = []
        for x_sample in x:
            x_sample = x_sample.flatten().detach().numpy()  # Flatten and convert to numpy for Pennylane

            # Forward pass through the quantum circuit
            q_out = quantum_circuit(x_sample)
            q_out = torch.tensor(q_out, dtype=torch.float32)  # Convert quantum output back to tensor
            quantum_output.append(q_out)

        quantum_output = torch.stack(quantum_output)  # Combine into a tensor
        return quantum_output

# Create an instance of the model
model = QuantumNet()

# Simulate input data
x = torch.rand(2, 128, 128, 3)  # Simulating a batch of 2 samples, with shape (128, 128, 3)

# Forward pass through the model
output = model(x)
print(output)


In [ ]:
import torch
import torch.nn as nn
import pennylane as qml
import torch.nn.functional as F

n_qubits = 128  # Set the number of qubits
dev = qml.device("default.qubit", wires=n_qubits)

# Define a quantum circuit
@qml.qnode(dev)
def quantum_circuit(x):
    """Process input through the quantum circuit."""
    x_flat = x.flatten()

    # Ensure the input matches the number of qubits
    if len(x_flat) != n_qubits:
        raise ValueError(f"Input size {len(x_flat)} does not match the number of qubits {n_qubits}")

    # Threshold the values to create a binary input
    binary_x = (x_flat > 0.5).astype(int)

    # BasisState preparation
    qml.BasisState(binary_x, wires=range(n_qubits))

    # Example quantum gates (can be modified)
    for i in range(n_qubits):
        qml.RX(binary_x[i], wires=i)

    # Return expectation value of PauliZ on the first qubit
    return qml.expval(qml.PauliZ(0))

# Define a hybrid quantum-classical neural network
class QuantumNet(nn.Module):
    def __init__(self):
        super(QuantumNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 16 * 16, n_qubits)  # Adjust for the flattened size

    def forward(self, x):
        # Permute the input to the correct format (batch_size, channels, height, width)
        x = x.permute(0, 3, 1, 2)

        # Preprocess the input
        x = self.pool(F.relu(self.conv1(x)))

        # Resize the image to match the number of qubits (e.g., 16x16)
        x = F.interpolate(x, size=(16, 16))  # Resize to 16x16

        # Flatten the image
        x = x.reshape(-1, 16 * 16 * 16)  # Use reshape instead of view
        x = F.relu(self.fc1(x))

        quantum_output = []
        for x_sample in x:
            x_sample = x_sample.flatten().detach().numpy()  # Flatten and convert to numpy for Pennylane

            # Forward pass through the quantum circuit
            q_out = quantum_circuit(x_sample)
            q_out = torch.tensor(q_out, dtype=torch.float32)  # Convert quantum output back to tensor
            quantum_output.append(q_out)

        quantum_output = torch.stack(quantum_output)  # Combine into a tensor
        return quantum_output

# Create an instance of the model
model = QuantumNet()

# Simulate input data
x = torch.rand(2, 128, 128, 3)  # Simulating a batch of 2 samples, with shape (128, 128, 3)

# Forward pass through the model
output = model(x)
print(output)


In [ ]:
import torch
import torch.nn as nn
import pennylane as qml
import torch.nn.functional as F

n_qubits = 128  # Set the number of qubits
dev = qml.device("default.qubit", wires=n_qubits)

# Define a quantum circuit
@qml.qnode(dev)
def quantum_circuit(x):
    """Process input through the quantum circuit."""
    # Ensure the input matches the number of qubits
    if len(x) != n_qubits:
        raise ValueError(f"Input size {len(x)} does not match the number of qubits {n_qubits}")

    # Threshold the values to create a binary input
    binary_x = (x > 0.5).astype(int)

    # BasisState preparation
    qml.BasisState(binary_x, wires=range(n_qubits))

    # Example quantum gates (can be modified)
    for i in range(n_qubits):
        qml.RX(binary_x[i], wires=i)

    # Return expectation value of PauliZ on the first qubit
    return qml.expval(qml.PauliZ(0))

# Define a hybrid quantum-classical neural network
class QuantumNet(nn.Module):
    def __init__(self):
        super(QuantumNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 16 * 16, n_qubits)  # Adjust for the flattened size

    def forward(self, x):
        # Permute the input to the correct format (batch_size, channels, height, width)
        x = x.permute(0, 3, 1, 2)

        # Preprocess the input
        x = self.pool(F.relu(self.conv1(x)))

        # Flatten the image and pass through the fully connected layer
        x = x.view(-1, 16 * 16 * 16)  # Reshape to (batch_size, 16*16*16)
        x = F.relu(self.fc1(x))

        # Initialize quantum output
        quantum_output = torch.zeros(x.size(0))

        # Forward pass through the quantum circuit for each sample in the batch
        for i in range(x.size(0)):
            x_sample = x[i].detach().numpy()  # Convert to numpy for Pennylane
            q_out = quantum_circuit(x_sample)  # Forward pass through the quantum circuit
            quantum_output[i] = torch.tensor(q_out, dtype=torch.float32)  # Store the result

        return quantum_output

# Create an instance of the model
model = QuantumNet()

# Simulate input data
x = torch.rand(2, 128, 128, 3)  # Simulating a batch of 2 samples, with shape (128, 128, 3)

# Forward pass through the model
output = model(x)
print(output)


In [ ]:
import torch
import torch.nn as nn
import pennylane as qml
import torch.nn.functional as F

n_qubits = 128  # Set the number of qubits
dev = qml.device("default.qubit", wires=n_qubits)

# Define a quantum circuit
@qml.qnode(dev)
def quantum_circuit(x):
    """Process input through the quantum circuit."""
    # Ensure the input matches the number of qubits
    if len(x) != n_qubits:
        raise ValueError(f"Input size {len(x)} does not match the number of qubits {n_qubits}")

    # Threshold the values to create a binary input
    binary_x = (x > 0.5).astype(int)

    # BasisState preparation
    qml.BasisState(binary_x, wires=range(n_qubits))

    # Example quantum gates (can be modified)
    for i in range(n_qubits):
        qml.RX(binary_x[i], wires=i)

    # Return expectation value of PauliZ on the first qubit
    return qml.expval(qml.PauliZ(0))

# Define a hybrid quantum-classical neural network
class QuantumNet(nn.Module):
    def __init__(self):
        super(QuantumNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 32 * 32, n_qubits)  # Adjust for the flattened size

    def forward(self, x):
        # Permute the input to the correct format (batch_size, channels, height, width)
        x = x.permute(0, 3, 1, 2)

        # Preprocess the input
        x = self.pool(F.relu(self.conv1(x)))

        # Flatten the image and pass through the fully connected layer
        x = x.view(x.size(0), -1)  # Reshape to (batch_size, features)
        x = F.relu(self.fc1(x))

        # Initialize quantum output
        quantum_output = torch.empty(x.size(0), dtype=torch.float32)  # Preallocate output tensor

        # Forward pass through the quantum circuit for each sample
        for i in range(x.size(0)):
            x_sample = x[i].detach().numpy()  # Convert to numpy for Pennylane

            # Forward pass through the quantum circuit
            q_out = quantum_circuit(x_sample)
            quantum_output[i] = torch.tensor(q_out, dtype=torch.float32)  # Convert quantum output back to tensor

        return quantum_output

# Create an instance of the model
model = QuantumNet()

# Simulate input data
x = torch.rand(2, 128, 128, 3)  # Simulating a batch of 2 samples, with shape (128, 128, 3)

# Forward pass through the model
output = model(x)
print(output)


In [ ]:
import torch
import torch.nn as nn
import pennylane as qml
import torch.nn.functional as F

n_qubits = 128  # Set the number of qubits
dev = qml.device("default.qubit", wires=n_qubits)

# Define a quantum circuit
@qml.qnode(dev)
def quantum_circuit(x):
    """Process input through the quantum circuit."""
    if len(x) != n_qubits:
        raise ValueError(f"Input size {len(x)} does not match the number of qubits {n_qubits}")

    binary_x = (x > 0.5).astype(int)
    qml.BasisState(binary_x, wires=range(n_qubits))

    for i in range(n_qubits):
        qml.RX(binary_x[i], wires=i)

    return qml.expval(qml.PauliZ(0))

# Define a hybrid quantum-classical neural network
class QuantumNet(nn.Module):
    def __init__(self):
        super(QuantumNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 64 * 64, n_qubits)  # Adjusted for the output size after pooling

    def forward(self, x):
        # Permute the input to the correct format (batch_size, channels, height, width)
        x = x.permute(0, 3, 1, 2)

        # Convolution and pooling
        x = self.pool(F.relu(self.conv1(x)))

        # Flatten the output and pass through the fully connected layer
        x = x.reshape(x.size(0), -1)  # Changed to reshape
        x = F.relu(self.fc1(x))

        # Initialize quantum output
        quantum_output = torch.empty(x.size(0), dtype=torch.float32)  # Preallocate output tensor

        # Forward pass through the quantum circuit for each sample
        for i in range(x.size(0)):
            x_sample = x[i].detach().numpy()  # Convert to numpy for Pennylane
            q_out = quantum_circuit(x_sample)  # Forward pass through the quantum circuit
            quantum_output[i] = torch.tensor(q_out, dtype=torch.float32)  # Convert back to tensor

        return quantum_output

# Create an instance of the model
model = QuantumNet()

# Simulate input data
x = torch.rand(2, 128, 128, 3)  # Simulating a batch of 2 samples, with shape (128, 128, 3)

# Forward pass through the model
output = model(x)
print(output)


In [ ]:
import torch
import torch.nn as nn
import pennylane as qml
import torch.nn.functional as F

n_qubits = 128  # Set the number of qubits
dev = qml.device("default.qubit", wires=n_qubits)

# Define a quantum circuit
@qml.qnode(dev)
def quantum_circuit(x):
    """Process input through the quantum circuit."""
    # Ensure the input is of size n_qubits
    if len(x) != n_qubits:
        raise ValueError(f"Input size {len(x)} does not match the number of qubits {n_qubits}")

    binary_x = (x > 0.5).astype(int)  # Convert to binary
    qml.BasisState(binary_x, wires=range(n_qubits))

    for i in range(n_qubits):
        qml.RX(binary_x[i], wires=i)

    return qml.expval(qml.PauliZ(0))

# Define a hybrid quantum-classical neural network
class QuantumNet(nn.Module):
    def __init__(self):
        super(QuantumNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 64 * 64, n_qubits)  # Adjusted for the output size after pooling

    def forward(self, x):
        # Permute the input to the correct format (batch_size, channels, height, width)
        x = x.permute(0, 3, 1, 2)

        # Convolution and pooling
        x = self.pool(F.relu(self.conv1(x)))

        # Flatten the output and pass through the fully connected layer
        x = x.reshape(x.size(0), -1)  # Changed to reshape
        x = F.relu(self.fc1(x))

        # Initialize quantum output
        quantum_output = torch.empty(x.size(0), dtype=torch.float32)  # Preallocate output tensor

        # Forward pass through the quantum circuit for each sample
        for i in range(x.size(0)):
            x_sample = x[i][:n_qubits].detach().numpy()  # Convert to numpy for Pennylane, limit to n_qubits
            q_out = quantum_circuit(x_sample)  # Forward pass through the quantum circuit
            quantum_output[i] = torch.tensor(q_out, dtype=torch.float32)  # Convert back to tensor

        return quantum_output

# Create an instance of the model
model = QuantumNet()

# Simulate input data
x = torch.rand(2, 128, 128, 3)  # Simulating a batch of 2 samples, with shape (128, 128, 3)

# Forward pass through the model
output = model(x)
print(output)


In [ ]:
# Function to prepare the dataset
def prepare_dataset(images_path, mask_path_train, mask_path_test):
    # Get lists of training and testing image paths
    images_train = list(images_path.glob('train/*.png'))  # Replace with your actual image file extension
    images_test = list(images_path.glob('test/*.png'))    # Replace with your actual image file extension

    # Get lists of training and testing mask paths
    masks_train = list(mask_path_train.glob('*.png'))  # Replace with your actual mask file extension
    masks_test = list(mask_path_test.glob('*.png'))    # Replace with your actual mask file extension

    return images_train, masks_train  # Return only the train images and masks

# Prepare dataset
IMAGES_PATH_TRAIN, MASK_PATH_TRAIN = prepare_dataset(
    IMAGES_PATH, MASK_PATH_TRAIN, MASK_PATH_TEST
)

# Assert lengths directly
assert len(IMAGES_PATH_TRAIN) == len(MASK_PATH_TRAIN), "Training images and masks count do not match."

print(type(IMAGES_PATH_TRAIN))  # Should be <class 'list'>
print(type(MASK_PATH_TRAIN))     # Should be <class 'list'>
print(len(IMAGES_PATH_TRAIN), len(MASK_PATH_TRAIN))

### Visualize an Image

Now that we have the data downloaded, we can proceed with actually visualizing some of the images and masks that we have.

Lets start by first seeing what one of these images looks like.

In [ ]:
img = PILImage.create(IMAGES_PATH_TRAIN.ls()[2])
img.show(figsize=(5, 5))

In [ ]:
# Check the number of images in the training set
num_images = len(IMAGES_PATH_TRAIN)
print(f"Number of training images: {num_images}")

# Use a valid index
if num_images > 2:  # Check if there are at least 3 images
    img_index = 2
    img = PILImage.create(IMAGES_PATH_TRAIN[img_index])  # Access the image by index
    img.show()  # Show the image
else:
    print("Not enough images available to access index 2.")



### Visualize a Mask

Next, to see the corresponding mask associated with our image, we'll need a small function to help us map them on
the fly. According to the AI4Mars info.txt file, the images end with extension `.JPG` while the corresponding label
ends with `.png`.

In [ ]:
get_mask_path = lambda file: MASK_PATH_TRAIN / f"{file.stem}.png"

In [ ]:
example_mask = PILMask.create(get_mask_path(IMAGES_PATH_TRAIN.ls()[2]))
example_mask.show()

### Data Engineering

In semantic segmentation, the “labels” are a 1:1 mask of the original picture with each pixel representing a label and are single channel:

In [ ]:
tensor(example_mask)

Very quickly we see an issue!

Because of how the loss gets calculated (and how fastai does things in general), the values of the pixel mask must be from 0 -> n, with n being the number of classes possible. If we take things as they are here during training you’ll hit an error that says “CUDA Segmentation Fault, Index Out of Bounds” (or something similar).

This is because our labels should be from 0 -> 4, to align with the fact predicted probabilities from our model are 0 -> 4. Instead they are 0, 1, 2, 3 and 255, leading to this issue.

So how do we fix the issue? In numpy we can just override the numbers for a particular value in the array and set it. To generalize this however a dictionary of the original value to the new one should also be made:

In [ ]:
unique_codes = {0: 0, 1: 1, 2: 2, 3: 3, 4: 255}

Next we need to create a get_y function. In this case it should take in a filename and our dictionary, open the filename, and return the mask:

In [ ]:
def get_label(filename: Path, unique_codes: dict) -> PILMask:
    filename = get_mask_path(filename)
    mask_array = np.asarray(PIL.Image.open(filename)).copy()

    mask_array[mask_array == 255] = 4

    return PILMask.create(mask_array)

Lets stop check this now.

In [ ]:
tensor(get_label(IMAGES_PATH_TRAIN.ls()[2], unique_codes))

In [ ]:
mask: PILMask = get_label(IMAGES_PATH_TRAIN.ls()[2], unique_codes)
mask.show()

Perfect!

The next thing we should do is create a list of our codes that map to the corresponding pixel value. According to the
info.txt, the codes are as follows:

| RGB         | Key             |
|-------------|-----------------|
| 0,0,0       | soil            |
| 1,1,1       | bedrock         |
| 2,2,2       | sand            |
| 3,3,3       | big rock        |
| 255,255,255 -> 4, 4, 4 | NULL (no label) |

In [ ]:
codes = np.array(["soil", "bedrock", "sand", "big rock", "null"], dtype=str)

## Create Our `DataLoader`

Now everything is in place to create our `DataBlock` object. Jeremy Howard, the founder of Fast.AI, popularized the idea of image resizing:

* Train on smaller sized images
* Eventually get larger and larger
* Transfer Learning loop

In the AI4Mars paper, the authors mention that they resize their images to 512x512 from the original 1024x1024 size.

Since I am training on a laptop GPU, I will resize the images to 256x256.

In [ ]:
mask_sizes = mask.shape
mask_sizes

In [ ]:
mask_sizes = tuple(int(x / 4) for x in mask_sizes)
mask_sizes

In the AI4Mars paper, the authors mention that the "batch size was chosen to be as large as possible before running into GPU memory issues". We will do the same.

In [ ]:
data_loader = DataBlock(
    blocks=(
        ImageBlock,
        MaskBlock(codes=codes),
    ),  # our input is an image and outpus is a mask
    splitter=RandomSplitter(),  # randomly split our dataset into 80% train and 20% valid
    get_y=partial(get_label, unique_codes=unique_codes),  # load our preprocessed labels
    batch_tfms=[
        *aug_transforms(size=mask_sizes),
        Normalize.from_stats(*imagenet_stats),
    ],  # apply some standard augs
).dataloaders(get_image_files(IMAGES_PATH_TRAIN), bs=8)

data_loader.show_batch()

## Select Our Model Architecture

Next, we can create our `Learner` object that will wrap our model architecture, hyperparameters, and `DataLoader` into one abstract object.

In the AI4Mars paper, the authors opted for the DeepLabv3+ model architecture with a ResNet-101 backend pretrained on ImageNet. For our architecture, we will go with a U-Net with a ResNet-34 backing that's been pre-trained on ImageNet. We _could_ utilize a fancy modern vision transformer, however, with spacecraft you want technology that is tried and true. U-Net has a long and rich history that make it a reliable model architecture for our purposes.

![U-Net Diagram](./../img/u_net_arch.png "U-Net Architecture Diagram")

Now lets define our U-Net architecture with the ResNet-18 backbone. The metrics
we use are the accuracy and mIoU. Our loss function is the CrossEntropyLoss.
The optimizer is good ol' Adam.

In [ ]:
cleanup_gpu_cache()

learner = unet_learner(
    data_loader,
    resnet18,
    metrics=[partial(foreground_acc, bkg_idx=4), JaccardCoeffMulti()],
    loss_func=CrossEntropyLossFlat(axis=1),
    opt_func=Adam,
    norm_type=None,
    wd_bn_bias=True,
)

This is what our model architecture looks like.

In [ ]:
learner.summary()

As per our requirements, our model **must** fit under the imposed limit. Lets check and make sure we are meeting our requirement.

In [ ]:
assert calc_model_size(learner.model) <= 200.0

## Train

The first thing we want to do before we train our model is find a good learning rate.

In [ ]:
learning_rates = learner.lr_find(suggest_funcs=(minimum, steep, valley, slide))

learner.lr = learning_rates.valley

f"Learning Rate: {learner.lr}"

Great! Lets now freeze our model. What freezing does is that it prevents layers of our model from changing
throughout the training process. This is very useful when we already have a pre-trained model whose existing
knowledge we don't want to lose. With freezing we freeze the layers from top down and the final layers are
free to learn the new segmentation task.

In [ ]:
# Lets freeze the top layers so that we don't lose some of the pre-trained
# knowledge
learner.freeze()

In [ ]:
# Set some callbacks for our model. One important callback is "early stopping"
# which is a simple regularization technique we can use to stop training when
# a specific metric, in this case the validation loss, stops improving.

# TODO: Have SaveModelCallback use safetensor instead of unsafe pickle format
callbacks: List[Callable] = [
    ShowGraphCallback(),
    EarlyStoppingCallback(),
    SaveModelCallback(fname="/workspace/models/ai4mars_model"),
]

if g_ENABLE_WANDB:
    callbacks.append(WandbCallback())

Finally, now we can train our model.

In [ ]:
cleanup_gpu_cache()

learner.fit_one_cycle(20, cbs=callbacks)

cleanup_gpu_cache()

Lets save the model to disk. Since we won’t be training the model further, we
can ignore the optimizer and only save the model weights.

In [ ]:
learner.save("/workspace/models/ai4mars_unet_resnet18", with_opt=False)

In [ ]:
learner.recorder.plot_metrics()

In [ ]:
learner.recorder.plot_sched()

## Evaluate

In [ ]:
learner.show_results(max_n=4)

In [ ]:
interpreter = SegmentationInterpretation.from_learner(learner)

In [ ]:
interpreter.plot_top_losses(4)

### Profile

In this section we want to profile our trained model to get a good feel for how
it will perform in the deployment environment.

In [ ]:
import torchvision

test_image = PIL.Image.open(IMAGES_PATH_UNUSED.ls()[3]).resize(mask_sizes)
test_image

In [ ]:
test_image = torchvision.transforms.functional.pil_to_tensor(test_image) / 255.0

test_image.unsqueeze_(0)
test_image.size()

In [ ]:
from torch.profiler import profile, record_function, ProfilerActivity

In [ ]:
learner.eval();

In [ ]:
learner.to("cpu")

with profile(
    activities=[ProfilerActivity.CPU], record_shapes=True, profile_memory=True
) as prof:
    with record_function("model_inference"):
        learner.predict(IMAGES_PATH_UNUSED.ls()[3])

In [ ]:
print(
    prof.key_averages(group_by_input_shape=True).table(
        sort_by="cpu_time_total", row_limit=10
    )
)

### Export

In [ ]:
learner.export("/workspace/models/ai4mars_unet_resnet18.pkl")

In [ ]:
from huggingface_hub import ModelCard, ModelCardData

In [ ]:
card_data = ModelCardData(
    language="en",
    license="mit",
    library_name="fastai",
    datasets=["ai4mars_v0.1"],
    base_model="U-Net ResNet-18 Backbone",
    metrics=["mIoU"],
)

card = ModelCard.from_template(
    card_data,
    model_id="ai4mars-unet-resnet18",
    model_description="semantic segmentation of the ai4mars terrain dataset",
    developers="Esteban Duran",
    repo="https://github.com/huggingface/huggingface_hub",
)
card.save("/workspace/models/ai4mars_model_card.md")
print(card)

## Conclusion and Future Work

We have sucessfully trained a model that meets all of our requirements. It
might not be the best and greatest but it works just fine under our constraints.

For some future work, I'd like to throw out some of these constraints and
explore what a state-of-the-art model would look like for this dataset. Perhaps
a transformer architecture? Additionally, it would be great to explore how to
use the data that was missing the labels. Perhaps we can manually label them,
although it will take some time and having an expert would be better. Or, we
could have a foundation model like SAM segment the image for us. Another
interesting avenue is to explore deployment optimization techniques like
quantization for efficient embedded device deployment.So many things to do!

## References

[1] AI4MARS: A dataset for Terrain-Aware autonomous driving on Mars. (2021, June 1). IEEE Conference Publication | IEEE Xplore. https://ieeexplore.ieee.org/document/9523149.

[2] SegFormer: Simple and Efficient Design for Semantic Segmentation with Transformers. (2021, October 21). https://arxiv.org/abs/2105.15203v3.

[2] Mish: A Self Regularized Non-Monotonic Activation Function. (2019, August 13). https://arxiv.org/abs/1908.08681